# TorchScript

In [1]:
import torch
print(torch.__version__)

1.12.0


## Basics of PyTorch Model Authoring

In [2]:
class MyCell(torch.nn.Module):
    def __init__(self):
        super(MyCell, self).__init__()

    def forward(self, x, h):
        new_h = torch.tanh(x + h)
        return new_h, new_h

my_cell = MyCell()
x = torch.rand(3, 4)
h = torch.rand(3, 4)
print(my_cell(x, h))

(tensor([[0.7247, 0.8846, 0.9132, 0.5198],
        [0.9375, 0.7856, 0.8612, 0.9073],
        [0.8522, 0.8196, 0.7847, 0.7808]]), tensor([[0.7247, 0.8846, 0.9132, 0.5198],
        [0.9375, 0.7856, 0.8612, 0.9073],
        [0.8522, 0.8196, 0.7847, 0.7808]]))


In [3]:
class MyCell(torch.nn.Module):
    def __init__(self):
        super(MyCell, self).__init__()
        self.linear = torch.nn.Linear(4, 4)

    def forward(self, x, h):
        new_h = torch.tanh(self.linear(x) + h)
        return new_h, new_h

my_cell = MyCell()
print(my_cell(x, h))

(tensor([[0.3679, 0.4809, 0.8279, 0.7401],
        [0.8164, 0.0168, 0.7751, 0.9260],
        [0.7121, 0.4753, 0.8263, 0.8745]], grad_fn=<TanhBackward0>), tensor([[0.3679, 0.4809, 0.8279, 0.7401],
        [0.8164, 0.0168, 0.7751, 0.9260],
        [0.7121, 0.4753, 0.8263, 0.8745]], grad_fn=<TanhBackward0>))


In [4]:
class MyDecisionGate(torch.nn.Module):
    def forward(self, x):
        if x.sum() > 0:
            return x
        else:
            return -x

class MyCell(torch.nn.Module):
    def __init__(self):
        super(MyCell, self).__init__()
        self.dg = MyDecisionGate()
        self.linear = torch.nn.Linear(4, 4)

    def forward(self, x, h):
        new_h = torch.tanh(self.dg(self.linear(x)) + h)
        return new_h, new_h

my_cell = MyCell()
print(my_cell)
print(my_cell(x, h))

MyCell(
  (dg): MyDecisionGate()
  (linear): Linear(in_features=4, out_features=4, bias=True)
)
(tensor([[-0.3702,  0.7388,  0.7840,  0.6857],
        [ 0.1624,  0.2606,  0.6917,  0.9130],
        [ 0.0595,  0.3753,  0.7872,  0.8733]], grad_fn=<TanhBackward0>), tensor([[-0.3702,  0.7388,  0.7840,  0.6857],
        [ 0.1624,  0.2606,  0.6917,  0.9130],
        [ 0.0595,  0.3753,  0.7872,  0.8733]], grad_fn=<TanhBackward0>))


## Basics of TorchScript

In [5]:
# Tracing Modules

class MyCell(torch.nn.Module):
    def __init__(self):
        super(MyCell, self).__init__()
        self.linear = torch.nn.Linear(4, 4)

    def forward(self, x, h):
        new_h = torch.tanh(self.linear(x) + h)
        return new_h, new_h

my_cell = MyCell()
# x = torch.rand(3, 4)
# h = torch.rand(3, 4)
traced_cell = torch.jit.trace(my_cell, (x, h))

print(traced_cell)
traced_cell(x, h)


MyCell(
  original_name=MyCell
  (linear): Linear(original_name=Linear)
)


(tensor([[-0.1901,  0.6948,  0.8473,  0.5471],
         [ 0.3855,  0.2973,  0.8437,  0.8405],
         [ 0.0389,  0.5929,  0.9210,  0.7361]], grad_fn=<TanhBackward0>),
 tensor([[-0.1901,  0.6948,  0.8473,  0.5471],
         [ 0.3855,  0.2973,  0.8437,  0.8405],
         [ 0.0389,  0.5929,  0.9210,  0.7361]], grad_fn=<TanhBackward0>))

In [6]:
print(traced_cell.graph)

graph(%self.1 : __torch__.MyCell,
      %x : Float(3, 4, strides=[4, 1], requires_grad=0, device=cpu),
      %h : Float(3, 4, strides=[4, 1], requires_grad=0, device=cpu)):
  %linear : __torch__.torch.nn.modules.linear.Linear = prim::GetAttr[name="linear"](%self.1)
  %20 : Tensor = prim::CallMethod[name="forward"](%linear, %x)
  %11 : int = prim::Constant[value=1]() # C:\Users\SB15\AppData\Local\Temp\ipykernel_51700\3908744505.py:9:0
  %12 : Float(3, 4, strides=[4, 1], requires_grad=1, device=cpu) = aten::add(%20, %h, %11) # C:\Users\SB15\AppData\Local\Temp\ipykernel_51700\3908744505.py:9:0
  %13 : Float(3, 4, strides=[4, 1], requires_grad=1, device=cpu) = aten::tanh(%12) # C:\Users\SB15\AppData\Local\Temp\ipykernel_51700\3908744505.py:9:0
  %14 : (Float(3, 4, strides=[4, 1], requires_grad=1, device=cpu), Float(3, 4, strides=[4, 1], requires_grad=1, device=cpu)) = prim::TupleConstruct(%13, %13)
  return (%14)



In [7]:
print(traced_cell.code)

def forward(self,
    x: Tensor,
    h: Tensor) -> Tuple[Tensor, Tensor]:
  linear = self.linear
  _0 = torch.tanh(torch.add((linear).forward(x, ), h))
  return (_0, _0)



In [8]:
print(my_cell(x, h))
print(traced_cell(x, h))

(tensor([[-0.1901,  0.6948,  0.8473,  0.5471],
        [ 0.3855,  0.2973,  0.8437,  0.8405],
        [ 0.0389,  0.5929,  0.9210,  0.7361]], grad_fn=<TanhBackward0>), tensor([[-0.1901,  0.6948,  0.8473,  0.5471],
        [ 0.3855,  0.2973,  0.8437,  0.8405],
        [ 0.0389,  0.5929,  0.9210,  0.7361]], grad_fn=<TanhBackward0>))
(tensor([[-0.1901,  0.6948,  0.8473,  0.5471],
        [ 0.3855,  0.2973,  0.8437,  0.8405],
        [ 0.0389,  0.5929,  0.9210,  0.7361]], grad_fn=<TanhBackward0>), tensor([[-0.1901,  0.6948,  0.8473,  0.5471],
        [ 0.3855,  0.2973,  0.8437,  0.8405],
        [ 0.0389,  0.5929,  0.9210,  0.7361]], grad_fn=<TanhBackward0>))


In [11]:
class MyDecisionGate(torch.nn.Module):
    def forward(self, x):
        if x.sum() > 0:
            return x
        else:
            return -x

class MyCell(torch.nn.Module):
    def __init__(self, dg):
        super(MyCell, self).__init__()
        self.dg = dg
        self.linear = torch.nn.Linear(4, 4)

    def forward(self, x, h):
        new_h = torch.tanh(self.dg(self.linear(x)) + h)
        return new_h, new_h

my_cell = MyCell(MyDecisionGate())
traced_cell = torch.jit.trace(my_cell, (x, h))

print(traced_cell.dg.code)
print(traced_cell.code)

def forward(self,
    argument_1: Tensor) -> Tensor:
  return torch.neg(argument_1)

def forward(self,
    x: Tensor,
    h: Tensor) -> Tuple[Tensor, Tensor]:
  dg = self.dg
  linear = self.linear
  _0 = torch.add((dg).forward((linear).forward(x, ), ), h)
  _1 = torch.tanh(_0)
  return (_1, _1)



C:\Users\SB15\AppData\Local\Temp\ipykernel_51700\4234398751.py:3: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if x.sum() > 0:


In [12]:
scripted_gate = torch.jit.script(MyDecisionGate())
my_cell = MyCell(scripted_gate)
scripted_cell = torch.jit.script(my_cell)

print(scripted_gate.code)
print(scripted_cell.code)

def forward(self,
    x: Tensor) -> Tensor:
  if bool(torch.gt(torch.sum(x), 0)):
    _0 = x
  else:
    _0 = torch.neg(x)
  return _0

def forward(self,
    x: Tensor,
    h: Tensor) -> Tuple[Tensor, Tensor]:
  dg = self.dg
  linear = self.linear
  _0 = torch.add((dg).forward((linear).forward(x, ), ), h)
  new_h = torch.tanh(_0)
  return (new_h, new_h)



In [13]:
# 새로운 입력
x, h = torch.rand(3, 4), torch.rand(3, 4)
traced_cell(x, h)

(tensor([[ 0.5360,  0.7507,  0.5911,  0.2068],
         [ 0.4522,  0.2178,  0.8173, -0.0303],
         [ 0.0432,  0.3520,  0.6772,  0.3201]], grad_fn=<TanhBackward0>),
 tensor([[ 0.5360,  0.7507,  0.5911,  0.2068],
         [ 0.4522,  0.2178,  0.8173, -0.0303],
         [ 0.0432,  0.3520,  0.6772,  0.3201]], grad_fn=<TanhBackward0>))

In [14]:
class MyRNNLoop(torch.nn.Module):
    def __init__(self):
        super(MyRNNLoop, self).__init__()
        self.cell = torch.jit.trace(MyCell(scripted_gate), (x, h))

    def forward(self, xs):
        h, y = torch.zeros(3, 4), torch.zeros(3, 4)
        for i in range(xs.size(0)):
            y, h = self.cell(xs[i], h)
        return y, h

rnn_loop = torch.jit.script(MyRNNLoop())
print(rnn_loop.code)

def forward(self,
    xs: Tensor) -> Tuple[Tensor, Tensor]:
  h = torch.zeros([3, 4])
  y = torch.zeros([3, 4])
  y0 = y
  h0 = h
  for i in range(torch.size(xs, 0)):
    cell = self.cell
    _0 = (cell).forward(torch.select(xs, 0, i), h0, )
    y1, h1, = _0
    y0, h0 = y1, h1
  return (y0, h0)



In [15]:
class WrapRNN(torch.nn.Module):
    def __init__(self):
        super(WrapRNN, self).__init__()
        self.loop = torch.jit.script(MyRNNLoop())

    def forward(self, xs):
        y, h = self.loop(xs)
        return torch.relu(y)

traced = torch.jit.trace(WrapRNN(), (torch.rand(10, 3, 4)))
print(traced.code)

def forward(self,
    xs: Tensor) -> Tensor:
  loop = self.loop
  _0, y, = (loop).forward(xs, )
  return torch.relu(y)

